In [1]:
import pandas as pd
df = pd.read_csv("df1.csv")
df = df.drop(columns="ID_material")
df = df.drop(columns="year-week")

In [2]:
df

,Units
0,120
1,120
2,300
3,60
4,120
...,...
294,180
295,180
296,180
297,0


In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from itertools import product

# Step 1: Load the data into a DataFrame
# df = ... (load your data into a pandas DataFrame)

# Step 2: Preprocessing (if required)

# Step 3: Split the data into training and testing sets
train_data = df.iloc[:-8]
test_data = df.iloc[-8:]

# Step 4: Grid search to find the best parameters for each model
# Set the parameter ranges to search through
p_values = range(0, 4)  # AR parameter range
d_values = range(0, 2)  # I(d) parameter range
q_values = range(0, 4)  # MA parameter range
seasonal_p_values = range(0, 4)  # Seasonal AR parameter range
seasonal_d_values = range(0, 4)  # Seasonal I(d) parameter range
seasonal_q_values = range(0, 4)  # Seasonal MA parameter range
s_values = range(0, 2)  # Seasonal period range

best_model = None
best_rmse = np.inf

# Grid search for ARIMA and SARIMA models
for p, d, q in product(p_values, d_values, q_values):
    try:
        model = ARIMA(train_data, order=(p, d, q))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=len(test_data))
        rmse = np.sqrt(mean_squared_error(test_data, forecast))
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model_fit
    except:
        continue

for p, d, q, s in product(p_values, d_values, q_values, s_values):
    try:
        model = SARIMAX(train_data, order=(p, d, q), seasonal_order=(p, d, q, s))
        model_fit = model.fit()
        forecast = model_fit.forecast(steps=len(test_data))
        rmse = np.sqrt(mean_squared_error(test_data, forecast))
        if rmse < best_rmse:
            best_rmse = rmse
            best_model = model_fit
    except:
        continue

# Step 5: Fit the best model on the training data
# The best_model should already be fitted from the grid search step

# Step 6: Generate forecasts for the test data
forecast = best_model.forecast(steps=len(test_data))

# Step 7: Evaluate the model using RMSE
rmse = np.sqrt(mean_squared_error(test_data, forecast))
print(f"RMSE: {rmse}")

c:\Users\Usuario\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\Usuario\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
c:\Users\Usuario\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\Usuario\anaconda3\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


RMSE: 84.79432771170397


c:\Users\Usuario\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [4]:
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  Units   No. Observations:                  291
Model:                 ARIMA(0, 1, 1)   Log Likelihood               -1712.062
Date:                Thu, 25 May 2023   AIC                           3428.125
Time:                        11:16:47   BIC                           3435.464
Sample:                             0   HQIC                          3431.065
                                - 291                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.9832      0.011    -87.239      0.000      -1.005      -0.961
sigma2      7762.1320    813.722      9.539      0.000    6167.266    9356.998
===================================================================================
Ljung-Box (L1) (Q):                  10.58   Jarque-Bera (JB):                11.02
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               1.46   Skew:                             0.28
Prob(H) (two-sided):                  0.07   Kurtosis:                         2.23
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [5]:
test_data.sum()

Units    1080
dtype: int64

In [6]:
forecast.sum()

962.6756940224652

In [7]:
from sklearn.metrics import mean_absolute_percentage_error
mean_absolute_percentage_error(test_data,forecast)

1.35484559277444e+17